# Implementation and Evaluation of Serverless Applications in the AWS Cloud Environment

## Structure 1 - 

## Structure 2 - Serverless WebApp

A web application using HTTP API will be deployed. This API includes various routes, such as `options`, `get`, `put`, and `delete`, corresponding to different paths. 

The setup includes a Lambda function and a DynamoDB table. To make it accessible to users, a static website hosted on Amazon S3 will be deployed. This static website will serve as the front-end interface for users, who can interact with the API via their web browsers.

The static website will contain a form where users can enter and submit information. The form will communicate with the public endpoint of the HTTP API, which will pass the data to the Lambda function. The Lambda function will process the request, and the processed data will be stored in a DynamoDB table.

1. A user interacts with the static website hosted on S3.
2. The website communicates with the HTTP API.
3. The API sends the request to the Lambda function.
4. The Lambda function processes the request and stores the result in the DynamoDB table.

The form that's generated will also be populated with information from the table, so it will always be returned back and viewable.

<img src="figs/serverless_webapp.png"  style="width:85%;"/>

### Create a Stack

On Amazon Cloud Formation, we create a Stack from an uploaded template `code/aws-backend-creation-cftemplate.yaml`

Stack name: `WebApp`

From this template, we can visualize the Application Composer canvas:

<img src="figs/application_composer_canvas.png"  style="width:85%;"/>

We need to check the field "I acknowledge that AWS CloudFormation might create IAM resources".

Click `Submit`.

### Create an S3 Bucket

On S3, create a bucket.

Bucket name: `frontend-website-00`

After creating the bucket, click on it and select "Properties". On "Static website hosting", click on "Edit". Select `Enable`.

Index document: `Index.html`.

Save changes.

Click on "Permissions". On "Block public access (bucket settings)", click on "Edit". Disable `Block all public access`.

Save changes.

Then Edit "Bucket policy". Insert the following code policy:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": "arn:aws:s3:::frontend-website-00/*"
        }
    ]
}

```

The Bucket ARN (`arn:aws:s3:::frontend-website-00`) is the field in "Resource" plus "/*".

The bucket policy, written in JSON, provides access to the objects stored in the bucket. Now we have a bucket policy, which allows GetObjects (that is, we can have public reads of objects within the bucket).

Go to "Cross-origin resource sharing (CORS)" and edit it. Insert this:

```json
[
    {
        "AllowedHeaders": [
            "*"
        ],
        "AllowedMethods": [
            "GET"
        ],
        "AllowedOrigins": [
            "*"
        ],
        "ExposeHeaders": []
    }
]
```

The CORS configuration, written in JSON, defines a way for client web applications that are loaded in one domain to interact with resources in a different domain. This CORS policy allows any headers, the `GET` method, and any origins.





### Configure API Gateway

Go to "API Gateway". You can see that Cloud Formation created an API called `items-api`. Click on it and go to "Stages". Click on "Create".

Name: `prod`.

Click on "create".

Go to "Integrations" > "Manage integrations" > "Create".

Attach this integration to a route: `$default`

Integration type: `Lambda function`

AWS Region: `us-east-1` (same for every region asked before)

Lambda function: select the option below

`arn:aws:lambda:us-east-1:093511299840:function:WebApp-ItemLambdaFunction-gphsBpHEDO06`

Click on "Create".

Copy the "Integration ID": `4gt0yqu`.

Click on "Routes" > "Create a route". We'll create different routes for different methods. In "Route and method", add the following:

`GET` -> `/items`

`PUT` -> `/items`

`OPTIONS` -> `/items`

`GET` -> `/items/{id}`

`DELETE` -> `/items/{id}`

`OPTIONS` -> `/items/{id}`

So we have:

```asciidoc
$default
├── /items
│   ├── PUT
│   ├── OPTIONS
│   ├── GET
│   └── /{id}
│       ├── OPTIONS
│       ├── GET
│       └── DELETE
```

Then we select each one of these, choose "Attach integration" > "Choose an existing integration". Then choose the one created before (remember Integration ID) and click on "Attach integration"

<img src="figs/routes_for_items_api.png"  style="width:15%;"/>

Go to "CORS" > "Configure".

Access-Control-Allow-Origin: `https://frontend-website-00.s3.amazonaws.com`

Access-Control-Allow-Headers: `*`

Access-Control-Allow-Methods: `*`

Access-Control-Expose-Headers: `*`

Access-Control-Max-Age: `96400`

Access-Control-Allow-Credentials: `yes`

Click on "Save". Click on "Deploy".

Stage: `prod`

In the "Stages" section in the left-hand panel, we can find `prod`'s invoke URL: `https://txi4irodvl.execute-api.us-east-1.amazonaws.com/prod`

### Client-side code

On our `code/client/src/config.ts` file, we need to have the invoke URL:

```typescript
const apiId = 'txi4irodvl'
export const apiEndpoint = `https://${apiId}.execute-api.us-east-1.amazonaws.com/prod`
```

Open a terminal.

```ssh
PS C:\Users\AlexFidalgoZamikhows\Projects\EACH\CloudComputing\TrabalhoFinal\code\client> npm install
...
Run `npm audit` for details.
npm notice
npm notice New major version of npm available! 9.8.1 -> 10.8.3
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.8.3
npm notice Run npm install -g npm@10.8.3 to update!
npm notice
```

Now run

```ssh
npm run build
```

Now we have a `build` directory with our compiled code in it:

```sh
PS C:\Users\AlexFidalgoZamikhows\Projects\EACH\CloudComputing\TrabalhoFinal\code\client> ls
    Directory: C:\Users\AlexFidalgoZamikhows\Projects\EACH\CloudComputing\TrabalhoFinal\code\client
Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----         9/30/2024  12:13 PM                build
d-----         9/30/2024  12:12 PM                node_modules
d-----         9/30/2024  11:46 AM                public
d-----         9/30/2024  11:46 AM                src
-a----         9/30/2024   9:02 AM            261 .editorconfig
-a----         9/30/2024   9:02 AM            107 .prettierrc.json
-a----         9/30/2024  11:57 AM         828178 package-lock.json
-a----         9/30/2024   9:02 AM           1090 package.json
-a----         9/30/2024   9:02 AM            791 README.md
-a----         9/30/2024   9:02 AM            494 tsconfig.json
```

Inside build:

```sh
    Directory: C:\Users\AlexFidalgoZamikhows\Projects\EACH\CloudComputing\TrabalhoFinal\code\client\build
Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----         9/30/2024  12:13 PM                static
-a----         9/30/2024  12:13 PM           1035 asset-manifest.json
-a----         9/30/2024   9:02 AM           3870 favicon.ico
-a----         9/30/2024  12:13 PM           2125 index.html
-a----         9/30/2024   9:02 AM            292 manifest.json
-a----         9/30/2024  12:13 PM           2503 precache-manifest.b82a456ee0bc7dc8666f0e1e42e9822f.js
-a----         9/30/2024  12:13 PM           1039 service-worker.js
```

### Upload files to S3 Bucket

Go to the `frontend-website-00` bucket. Under "Objects" select "Upload". Paste all documents inside the `build` directory and click on "Upload".

Under "Objects", click on `index.html` and copy:

Object URL: `https://frontend-website-00.s3.amazonaws.com/index.html`

### Accessing and Using WebApp

Open a browser and go to `https://frontend-website-00.s3.amazonaws.com/index.html`.

<img src="figs/webapp_index.png"  style="width:85%;"/>

We have a form displayed. If we click on it, we see that no items have been added yet:

<img src="figs/items.png"  style="width:25%;"/>

We can add items:

<img src="figs/adding_item.png"  style="width:25%;"/>

We got an error when trying to add a new item:

``` css
Failed to fetch items: Request failed with status code 500
```

Let's check the "logs" in `CloudWatch`:

<img src="figs/error_logs.png"  style="width:85%;"/>

If we expand the ERROR log, we get:

```bash
2024-09-30T16:05:06.899Z	undefined	ERROR	Uncaught Exception 	{
    "errorType": "Runtime.ImportModuleError",
    "errorMessage": "Error: Cannot find module 'aws-sdk'\nRequire stack:\n- /var/task/index.js\n- /var/runtime/index.mjs",
    "stack": [
        "Runtime.ImportModuleError: Error: Cannot find module 'aws-sdk'",
        "Require stack:",
        "- /var/task/index.js",
        "- /var/runtime/index.mjs",
        "    at _loadUserApp (file:///var/runtime/index.mjs:1087:17)",
        "    at async UserFunction.js.module.exports.load (file:///var/runtime/index.mjs:1119:21)",
        "    at async start (file:///var/runtime/index.mjs:1282:23)",
        "    at async file:///var/runtime/index.mjs:1288:1"
    ]
}
```

The module "aws-sdk" hadn't been installed from the template, so we had to do it manually.

Now we can successfully add items:

<img src="figs/filled_form.png"  style="width:70%;"/>

Now we check on `DynamoDB` if they are actually there. Go to "DynamoDB" > "Tables", click on the `all-items` table, click on "Explore items".

<img src="figs/dynamodb_items.png"  style="width:60%;"/>

Likewise, if we delete items on the frontend, they are deleted from the DynamoDB database.

After finished, remove the created Stack (this will delete the API Gateway, the Lambda function and the DynamoDB table) and then remove the S3 Bucket.